# A Cando-app to setup free energy perturbation calculations using AMBER

First we do the following:

  * Load the TIRUN package.
  * Set things up to carry out geometry optimization using Amber.

In [ ]:
*default-pathname-defaults*

In [ ]:
(ql:quickload :tirun)

In [ ]:
(setup-default-paths)

In [ ]:
(load-atom-type-rules "ATOMTYPE_GFF.DEF")

In [ ]:
(clasp-posix:getpid)

In [ ]:
source "leaprc.ff14SB.redq"

In [ ]:
source "leaprc.gaff"

In [ ]:
(list-force-fields)

In [ ]:
(start-swank)

# Start a new TIRUN calculation

In [ ]:
(defparameter *tiruns* (make-instance 'tirun::tirun-calculation))

# Import sketch - use Chemdraw

## Open chemdraw and add more functional groups if you want

In [ ]:
(png-from-file "ligands.png" )

In [ ]:
(defparameter *sk* (handler-bind ((warning #'muffle-warning))
             (with-open-file (fin (open "ligands.cdxml" :direction :input))
             (chem:make-chem-draw fin :add-hydrogens nil))))

In [ ]:
(tirun:setup-ligands *tiruns* *sk*)

In [ ]:
(defparameter *v* (show (tirun:layout-ligands *tiruns*))) *v*

In [ ]:
(defparameter *tests* (list (cons :c1 (lambda (a) (eq (chem:get-name a) :c1)))
                  (cons :c3 (lambda (a) (eq (chem:get-name a) :c3)))
                  (cons :c5 (lambda (a) (eq (chem:get-name a) :c5)))))

In [ ]:
(defparameter *pick* (chem:compile-smarts 
             "[C:6]1~[C<c1>:1]~[C:2]~[C<c3>:3]~[C:4]~[C<c5>:5]1" :tests *tests*))

In [ ]:
(show (tirun:molecule (find-if (lambda (x) (eq (tirun:name x) :AA)) (tirun:ligands *tiruns*))))

# Load the lysozyme PDB file

In [ ]:
(defparameter *lysozyme* (load-pdb "181L_mod.pdb"))

In [ ]:
(cando:build-unbuilt-hydrogens *lysozyme*)

In [ ]:
(simple-build-unbuilt-atoms *lysozyme*)

In [ ]:
(defparameter *v* (show *lysozyme*)) *v*

In [ ]:
(nglv::add-representation *v* "ball+stick" :selection "all")

In [ ]:
(pushnew *lysozyme* (tirun:receptors *tiruns*))

# Load the ligands

In [ ]:
*default-pathname-defaults*

In [ ]:
(load-off "phen.lib")
(load-off "benz.lib")

In [ ]:
(defparameter *ligs* (load-pdb "bnz_phn.pdb"))

In [ ]:
(simple-build-unbuilt-atoms *ligs*)
(build-unbuilt-hydrogens *ligs*)

In [ ]:
(show *ligs*)

## Print the fixed atoms of the template ligand

In [ ]:
(alexandria:hash-table-alist (tirun:pattern-atoms *pick* *ligs*))

# Use the GAFF force field to build chemically reasonable structures of candidate ligands

In [ ]:
(tirun:build-ligands *tiruns*)

In [ ]:
(mapcar #'tirun:molecule (tirun:ligands *tiruns*))

In [ ]:
(show (tirun:layout-ligands *tiruns* :accessor 'tirun::molecule))

# Pose the new ligands onto the template ligand

In [ ]:
(tirun::pose-ligands-using-pattern *tiruns* *pick* *ligs*)

In [ ]:
(defparameter vreceptor (show *lysozyme*)) vreceptor

In [ ]:
(defparameter *moveable-agg* (chem:make-aggregate))
(chem:add-matter *moveable-agg* (tirun::molecule (second (tirun:ligands *tiruns*))))
(defparameter *struct* (make-instance 'cando-structure :matter *moveable-agg*))
(nglv::add-structure vreceptor *struct*)

# Define the pairs of compounds between which we want to carry out free energy perturbation calculations

In [ ]:
(tirun:build-job-nodes *tiruns*)

In [ ]:
(tirun::connect-job-nodes *tiruns* :simple :connections 2 :stages 3 :windows 11)

In [ ]:
(tirun:jobs *tiruns*)

In [ ]:
(graph::save-graph *tiruns*)

# Generate the calculations

In [ ]:
(defparameter *work-list* (tirun::generate-jobs *tiruns*))

In [ ]:
*work-list*

In [ ]:
(with-open-file (sout "/tmp/graph.dot" :direction :output)
    (tirundot::draw-graph-stream (list *work-list*) sout))

In [ ]:
(ext:system "dot -Tpdf -O /tmp/graph.dot")

In [ ]:
(ext:system "open -n /tmp/graph.dot.pdf")